tap으로 send, reply가 구분되어 있는 pair corpus를 불러올 수 있는 클래스를 하나 만듧니다. 

cohesion 학습을 위하여 pair 형태로 출력할지, 1 문장 형태로 출력할 지 정할 수 있는 yield_pair argument를 만듦니다

In [1]:
class PairCorpus:
    def __init__(self, fname, yield_pair=True):
        self.fname = fname
        self.yield_pair = yield_pair
        
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                try:
                    send, reply = doc.replace('\n','').split('\t')
                    if self.yield_pair:
                        yield (send, reply)
                    else:
                        for sent in [send, reply]:
                            yield sent
                except Exception as e:
                    print(e)
                    break
                    
corpus_fname = '../data/raw/animation/naruto.txt'
corpus = PairCorpus(corpus_fname, yield_pair=False)
for num, pair in enumerate(corpus):
    if num > 10: break
    print(pair)

어떠냐
이걸 네가 빌린 사륜안으로 없앨 수 있겠어
이걸 네가 빌린 사륜안으로 없앨 수 있겠어
진짜와 빌려쓰는 것의 차이를 확실히 보여주지
진짜와 빌려쓰는 것의 차이를 확실히 보여주지
젠장
젠장
아 앞이
아 앞이
사쿠라 어째서 여기에 온거야
사쿠라 어째서 여기에 온거야


### Cohesion probability 학습

MaxScoreTokenizer에 이용될 scores를 만듦니다

In [2]:
from cohesion import CohesionProbability

cohesion = CohesionProbability()
cohesion.train(corpus)

In [3]:
scores = cohesion.get_all_cohesion_probabilities()

scores는 {word: (cohesion_l, cohesion_r, frequency_l, frequency_r} 형태로 출력되므로, 우리가 토크나이저에 이용할 cohesion_l 점수만 선택합니다. 

cohesion이 0.01이 넘고 빈도수가 10번 이상인 subwords에 대해서만 scores를 만들었습니다. 

In [4]:
print('before |scores| = %d' % len(scores))
scores = {word:s[0] for word, s in scores.items() if (s[0] > 0.01 and s[2] > 10)}
print('after |scores| = %d' % len(scores))

before |scores| = 270262
after |scores| = 53803


In [5]:
import pickle

with open('./conversationbot/naruto_scores.pkl', 'wb') as f:
    pickle.dump(scores, f)

### MaxScoreTokenizer 만들기

In [6]:
from tokenizer import MaxScoreTokenizer

tokenizer = MaxScoreTokenizer(scores=scores)
tokenizer.tokenize('나루토이건무슨일이야사스케는어디있어')

['나루토', '이건', '무슨', '일이', '야', '사스케', '는', '어디', '있어']

### Similar sends를 tokenization 시킨 코퍼스 만들기

In [7]:
corpus.yield_pair = True
with open('./data/naruto_send_tokenized_corpus.txt', 'w', encoding='utf-8') as f:
    for send, reply in corpus:
        send = ' '.join(tokenizer.tokenize(send))
        f.write('%s\t%s\n' % (send, reply))

## Chatbot 구성요소 학습하기

### send2reply

In [8]:
corpus_fname = './data/naruto_send_tokenized_corpus.txt'
corpus = PairCorpus(corpus_fname)

for num, pair in enumerate(corpus):
    if num > 5: break
    print(pair)

('어떠냐', '이걸 네가 빌린 사륜안으로 없앨 수 있겠어')
('이걸 네가 빌린 사륜안 으로 없앨 수 있 겠어', '진짜와 빌려쓰는 것의 차이를 확실히 보여주지')
('진짜 와 빌려 쓰는 것의 차이를 확실 히 보여주 지', '젠장')
('젠장', '아 앞이')
('아 앞이', '사쿠라 어째서 여기에 온거야')
('사쿠라 어째서 여기 에 온거야', '그만 둬')


In [9]:
%%time
from send2reply import Send2Reply
send2reply = Send2Reply()
send2reply.train(corpus)

CPU times: user 1.55 s, sys: 44 ms, total: 1.59 s
Wall time: 1.58 s


In [10]:
send2reply.save('./conversationbot/naruto_send2reply')

In [11]:
print('# sends = %d, # replys = %d' % (len(send2reply.send_set), len(send2reply.reply_set)))

# sends = 112894, # replys = 118369


### Vectorizer와 send x 만들어서 Indexer에 넣기

In [12]:
from vectorizer import Vectorizer
vectorizer = Vectorizer(tokenizer=tokenizer.tokenize)
send_x = vectorizer.fit_transform(send2reply.send_set)

In [13]:
from indexer import FullSearchIndexer

indexer = FullSearchIndexer(send_x)
indexer.save('./conversationbot/naruto_index')

### Chatbot 구성요소를 모두 넣어 나루토봇 만들기

In [14]:
from conversation_chatbot import ConversationBot

conversation_bot = ConversationBot(vectorizer, indexer, send2reply)

In [15]:
conversation_bot.process('우린 친구니까', n_similar_sends=5, n_reply=10)

[('무 무슨 소릴 하는거야 녀석과는 친구가 아니야', 5),
 ('사스케', 4),
 ('그렇지 않다니까', 2),
 ('그렇구나', 2),
 ('무슨 소리하는거야 별로 그녀석은 친구따위는 아니야', 2),
 ('너 설마 도와줄 셈이야', 1),
 ('도망칠수가 없잖아', 1)]

In [16]:
conversation_bot.process('나루토 사스케를 부탁해', n_similar_sends=5, n_reply=10)

[('누구맘대로', 17),
 ('안돼 하타케 카카시는 관여하지마', 17),
 ('데려와줘', 17),
 ('사스케를 데리고 와줘', 9),
 ('나루토는 스스로 구미를 억눌렀어요', 7),
 ('사스케를 다시 꺼내라', 6),
 ('우치하 사스케를 말살하는 걸 그만 둬 줬으면 해', 6),
 ('사스케를 데리고 와 줘', 4),
 ('사스케를', 4),
 ('어서 놓아줘', 3)]

In [17]:
conversation_bot.process('오랜만이군', n_similar_sends=5, n_reply=10)

[('우치하 사건 이후로 처음인가', 8),
 ('이쪽도 분신인가', 4),
 ('너 아카츠키를 수색중이지 않았던가', 2),
 ('사스케', 1),
 ('카카시의 사륜안은 내가 접수하겠어', 1),
 ('성장한 건 그 덩치 뿐이냐', 1)]